## StarTracker Coordinate Check - 08-Mar-23

Craig Lage - Double checking coordinate transformations

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

In [ ]:
print(SIMONYI_LOCATION.lat.deg, SIMONYI_LOCATION.lon.deg)

In [ ]:
tcsLocation = EarthLocation.from_geodetic(
            lon=-70.747698 * u.deg, lat=-30.244728 * u.deg, height=2663.0 * u.m)

In [ ]:
(SIMONYI_LOCATION.lon.deg - tcsLocation.lon.deg) * 3600.0

In [ ]:
(SIMONYI_LOCATION.lat.deg - tcsLocation.lat.deg) * 3600.0

In [ ]:
def  readRubinTV_json(date, camera):
    year = int(date/10000)
    month = int((date - 10000 * year)/100)
    day = int((date - 10000 * year - 100 * month))
    if camera == 'Wide':
        filename = f'/scratch/cslage/starTracker/startracker-wide_{year}-{month:02}-{day:02}.json'
    elif camera == 'Narrow':
        filename = f'/scratch/cslage/starTracker/startracker_{year}-{month:02}-{day:02}.json'
    elif camera == 'AuxTel':
        filename = f'/scratch/cslage/starTracker/auxtel_{year}-{month:02}-{day:02}.json'
    df = pd.read_json(filename)
    df = df.transpose()
    print(filename)
    return df

In [ ]:
[camera, num] = ['Narrow', 102]
date = 20230307
year = int(date/10000)
month = int((date - 10000 * year)/100)
day = int((date - 10000 * year - 100 * month))
df = readRubinTV_json(date, camera)

In [ ]:
row = df.loc[11]
# First check the header values
# First go from RA/Dec to AltAz
skyLocation = SkyCoord(row['Ra']*u.deg, row['Dec']*u.deg)
time = Time(row['MJD'], format='mjd', scale='tai')
altAz1 = AltAz(obstime=time, location=SIMONYI_LOCATION)
obsAltAz1 = skyLocation.transform_to(altAz1)
print(f"Header AltAz values =             ({row['Alt']}, {row['Az']})")
print(f"AltAz from header Ra/Dec values = ({obsAltAz1.alt.deg}, {obsAltAz1.az.deg})")
# Now go the other way
altAz2 = SkyCoord(AltAz(alt=row['Alt']*u.deg, az=row['Az']*u.deg, obstime=time, location=SIMONYI_LOCATION))
RaDec = altAz2.transform_to(ICRS)
print(f"Header RA/Dec values =            ({row['Ra']}, {row['Dec']})")
print(f"RA/Dec from header AltAz values = ({RaDec.ra.deg}, {RaDec.dec.deg})")
print()

# Now check the calculated values
# First go from RA/Dec to AltAz
skyLocation = SkyCoord(row['Calculated Ra']*u.deg, row['Calculated Dec']*u.deg)
time = Time(row['MJD'], format='mjd', scale='tai')
altAz1 = AltAz(obstime=time, location=SIMONYI_LOCATION)
obsAltAz1 = skyLocation.transform_to(altAz1)
print(f"Calculated AltAz values =             ({row['Calculated Alt']}, {row['Calculated Az']})")
print(f"AltAz from calculated Ra/Dec values = ({obsAltAz1.alt.deg}, {obsAltAz1.az.deg})")
# Now go the other way
altAz2 = SkyCoord(AltAz(alt=row['Calculated Alt']*u.deg, az=row['Calculated Az']*u.deg, obstime=time, location=SIMONYI_LOCATION))
RaDec = altAz2.transform_to(ICRS)
print(f"Calculated RA/Dec values =            ({row['Calculated Ra']}, {row['Calculated Dec']})")
print(f"RA/Dec from calculated AltAz values = ({RaDec.ra.deg}, {RaDec.dec.deg})")


In [ ]:
row = df.loc[11]
# Now check the calculated values
# First go from RA/Dec to AltAz
skyLocation = SkyCoord(row['Calculated Ra']*u.deg, row['Calculated Dec']*u.deg)
time = Time(row['MJD'], format='mjd', scale='tai')
altAz1 = AltAz(obstime=time, location=SIMONYI_LOCATION)
obsAltAz1 = skyLocation.transform_to(altAz1)
print(f"Calculated AltAz values =             ({row['Calculated Alt']}, {row['Calculated Az']})")
print(f"AltAz from calculated Ra/Dec values = ({obsAltAz1.alt.deg}, {obsAltAz1.az.deg})")
deltaAlt = (row['Calculated Alt'] - obsAltAz1.alt.deg) * 3600.0
deltaAz = (row['Calculated Az'] - obsAltAz1.az.deg) * 3600.0
print(f"DeltaAlt = {deltaAlt:.1f} arcseconds, DeltaAz = {deltaAz:.1f} arcseconds")
print()

# Now go the other way
altAz2 = SkyCoord(AltAz(alt=row['Calculated Alt']*u.deg, az=row['Calculated Az']*u.deg, obstime=time, location=SIMONYI_LOCATION))
RaDec = altAz2.transform_to(ICRS)
print(f"Calculated RA/Dec values =            ({row['Calculated Ra']}, {row['Calculated Dec']})")
print(f"RA/Dec from calculated AltAz values = ({RaDec.ra.deg}, {RaDec.dec.deg})")
deltaRA = (row['Calculated Ra'] - RaDec.ra.deg) * 3600.0
deltaDec = (row['Calculated Dec'] - RaDec.dec.deg) * 3600.0
print(f"DeltaRA = {deltaRA:.1f} arcseconds, DeltaDec = {deltaDec:.1f} arcseconds")


In [ ]:
camera = 'AuxTel'
date = 20230302
year = int(date/10000)
month = int((date - 10000 * year)/100)
day = int((date - 10000 * year - 100 * month))
df = readRubinTV_json(date, camera)

In [ ]:
row = df.loc[151]
# First check the header values
# First go from RA/Dec to AltAz
skyLocation = SkyCoord(row['RA']*u.deg, row['Dec']*u.deg)
time = Time(f"{year}-{month:02}-{day:02}T{row['TAI']}", scale='tai')
altAz1 = AltAz(obstime=time, location=AUXTEL_LOCATION)
obsAltAz1 = skyLocation.transform_to(altAz1)
print("AuxTel")
print(f"Header AltAz values =             ({row['Altitude']}, {row['Azimuth']})")
print(f"AltAz from header Ra/Dec values = ({obsAltAz1.alt.deg}, {obsAltAz1.az.deg})")


In [ ]:
from lsst.daf.butler import Butler
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])

In [ ]:
expId = 2023030200099
mData = butler.get('raw.metadata', detector=0, exposure=expId)
skyLocation = SkyCoord(mData['RASTART']*u.deg, mData['DECSTART']*u.deg)
time = Time(mData['DATE-BEG'], scale='tai')
altAz1 = AltAz(obstime=time, location=AUXTEL_LOCATION)
obsAltAz1 = skyLocation.transform_to(altAz1)
print(f"AuxTel - {expId}")
print(f"Header AltAz values =             ({mData['ELSTART']}, {mData['AZSTART']})")
print(f"AltAz from header Ra/Dec values = ({obsAltAz1.alt.deg}, {obsAltAz1.az.deg})")


In [ ]:
expId = 2023030200099
exp = butler.get('raw', detector=0, exposure=expId)
wcs = exp.getWcs()
print(wcs.getSkyOrigin())
print(mData['RASTART'], mData['DECSTART'])

In [ ]:
for expId in [2023030200099, 2023030200151]:
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    time = Time(mData['DATE-BEG'], scale='tai')
    print(f"{expId}, DATE-BEG = {time.isot}")

In [ ]:
bogusLocation=EarthLocation.from_geodetic(
            lon=-70.749417 * u.deg, lat=-30.244639 * u.deg, height=2665.0 * u.m)
#-30.244639 -70.749417

In [ ]:
expId = 2023030200099
mData = butler.get('raw.metadata', detector=0, exposure=expId)
skyLocation = SkyCoord(mData['RASTART']*u.deg, mData['DECSTART']*u.deg, frame=FK5, equinox='J2022.25')
time = Time(mData['DATE-BEG'], scale='tai') + TimeDelta(0.0, format='sec')
altAz1 = AltAz(obstime=time, location=AUXTEL_LOCATION)
obsAltAz1 = skyLocation.transform_to(altAz1)
print(f"AuxTel - {expId}")
print(f"Header AltAz values =             ({mData['ELSTART']}, {mData['AZSTART']})")
print(f"AltAz from header Ra/Dec values = ({obsAltAz1.alt.deg}, {obsAltAz1.az.deg})")


In [ ]:
filename = '/scratch/cslage/starTracker/startracker_2023-03-09.json'
df = pd.read_json(filename)
df = df.transpose()

row = df.loc[1798]
print("StarTracker Narrow 2023-03-09 SeqNum 1798")
print()
for frame in [ICRS, FK5(equinox='J2023')]:
    print(f"Frame = {frame}")
    print()
    skyLocation = SkyCoord(row['Ra']*u.deg, row['Dec']*u.deg, frame=frame)
    time = Time(row['MJD'], format='mjd', scale='tai')
    altAz1 = AltAz(obstime=time, location=SIMONYI_LOCATION)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    print(f"Header AltAz values =             ({row['Alt']}, {row['Az']})")
    print(f"AltAz from header Ra/Dec values = ({obsAltAz1.alt.deg}, {obsAltAz1.az.deg})")
    deltaAlt = (row['Alt'] - obsAltAz1.alt.deg) * 3600.0
    deltaAz = (row['Az'] - obsAltAz1.az.deg) * 3600.0
    print(f"DeltaAlt = {deltaAlt:.1f} arcseconds, DeltaAz = {deltaAz:.1f} arcseconds")
    print()
    # Now go the other way
    altAz2 = SkyCoord(AltAz(alt=row['Alt']*u.deg, az=row['Az']*u.deg, obstime=time, location=SIMONYI_LOCATION))
    RaDec = altAz2.transform_to(frame)
    print(f"Header RA/Dec values =            ({row['Ra']}, {row['Dec']})")
    print(f"RA/Dec from header AltAz values = ({RaDec.ra.deg}, {RaDec.dec.deg})")
    deltaRA = (row['Ra'] - RaDec.ra.deg) * 3600.0
    deltaDec = (row['Dec'] - RaDec.dec.deg) * 3600.0
    print(f"DeltaRA = {deltaRA:.1f} arcseconds, DeltaDec = {deltaDec:.1f} arcseconds")
    print()


In [ ]:
#Why is there still a ~10 arcsecond discrepancy?

filename = '/scratch/cslage/starTracker/startracker_2023-03-09.json'
df = pd.read_json(filename)
df = df.transpose()
location = tcsLocation
row = df.loc[1798]
print("StarTracker Narrow 2023-03-09 SeqNum 1798")
print()
for frame in [ICRS, FK5(equinox='J2023')]:
    print(f"Frame = {frame}")
    print()
    skyLocation = SkyCoord(row['Ra']*u.deg, row['Dec']*u.deg, frame=frame)
    time = Time(row['MJD'], format='mjd', scale='tai') + TimeDelta(-2.0, format='sec')
    altAz1 = AltAz(obstime=time, location=location)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    print(f"Header AltAz values =             ({row['Alt']}, {row['Az']})")
    print(f"AltAz from header Ra/Dec values = ({obsAltAz1.alt.deg}, {obsAltAz1.az.deg})")
    deltaAlt = (row['Alt'] - obsAltAz1.alt.deg) * 3600.0
    deltaAz = (row['Az'] - obsAltAz1.az.deg) * 3600.0
    print(f"DeltaAlt = {deltaAlt:.1f} arcseconds, DeltaAz = {deltaAz:.1f} arcseconds")
    print()
    # Now go the other way
    altAz2 = SkyCoord(AltAz(alt=row['Alt']*u.deg, az=row['Az']*u.deg, obstime=time, location=location))
    RaDec = altAz2.transform_to(frame)
    print(f"Header RA/Dec values =            ({row['Ra']}, {row['Dec']})")
    print(f"RA/Dec from header AltAz values = ({RaDec.ra.deg}, {RaDec.dec.deg})")
    deltaRA = (row['Ra'] - RaDec.ra.deg) * 3600.0
    deltaDec = (row['Dec'] - RaDec.dec.deg) * 3600.0
    print(f"DeltaRA = {deltaRA:.1f} arcseconds, DeltaDec = {deltaDec:.1f} arcseconds")
    print()


In [ ]:
expId = 2023030200099
mData = butler.get('raw.metadata', detector=0, exposure=expId)
print(f"AuxTel - {expId}")
print()

for frame in [ICRS, FK5(equinox='J2023')]:
    print(f"Frame = {frame}")
    skyLocation = SkyCoord(mData['RASTART']*u.deg, mData['DECSTART']*u.deg, frame=frame)
    time = Time(mData['DATE-BEG'], scale='tai') + TimeDelta(0.0, format='sec')
    altAz1 = AltAz(obstime=time, location=AUXTEL_LOCATION)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    print(f"Header AltAz values =             ({mData['ELSTART']}, {mData['AZSTART']})")
    print(f"AltAz from header Ra/Dec values = ({obsAltAz1.alt.deg}, {obsAltAz1.az.deg})")
    deltaAlt = (mData['ELSTART'] - obsAltAz1.alt.deg) * 3600.0
    deltaAz = (mData['AZSTART'] - obsAltAz1.az.deg) * 3600.0
    print(f"DeltaAlt = {deltaAlt:.1f} arcseconds, DeltaAz = {deltaAz:.1f} arcseconds")
    print()


In [ ]:
filename = '/scratch/cslage/starTracker/startracker_2023-03-07.json'
df = pd.read_json(filename)
df = df.transpose()
row = df.loc[11]

icrs = ICRS; fk5 = FK5(equinox='J2023')
skyLocation = SkyCoord(row['Calculated Ra']*u.deg, row['Calculated Dec']*u.deg, frame=icrs)
time = Time(row['MJD'], format='mjd', scale='tai')
altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
obsAltAz1 = skyLocation.transform_to(altAz)

skyLocation2 = skyLocation.transform_to(fk5)
obsAltAz2 = skyLocation2.transform_to(altAz)
deltaAlt = (obsAltAz2.alt.deg - obsAltAz1.alt.deg) * 3600.0
deltaAz = (obsAltAz2.az.deg - obsAltAz1.az.deg) * 3600.0
print(f"DeltaAlt = {deltaAlt:.1f} arcseconds, DeltaAz = {deltaAz:.1f} arcseconds")
# Surprisingly at first, this is zero.  However, thinking some more,
# it says that the same location on the sky is at the same AltAz
# regardless of which coordinate system you use.

In [ ]:
filename = '/scratch/cslage/starTracker/startracker_2023-03-07.json'
df = pd.read_json(filename)
df = df.transpose()
row = df.loc[27]

icrs = ICRS; fk5 = FK5(equinox='J2023')
time = Time(row['MJD'], format='mjd', scale='tai')
skyLocation = SkyCoord(row['Ra']*u.deg, row['Dec']*u.deg, frame=icrs)
altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
obsAltAz1 = skyLocation.transform_to(altAz)
deltaAlt = (row['Calculated Alt'] - obsAltAz1.alt.deg) * 3600.0
deltaAz = (row['Calculated Az'] - obsAltAz1.az.deg) * 3600.0
print(f"{seqNum}: DeltaAlt = {deltaAlt:.1f} arcseconds, DeltaAz = {deltaAz:.1f} arcseconds")

In [ ]:
filename = '/scratch/cslage/starTracker/startracker_2023-03-09.json'
df = pd.read_json(filename)
df = df.transpose()
df = df[~df.isnull().any(axis=1)]
alts = []
azs = []
deltaAlts = []
deltaAzs = []

for seqNum in df.index.values.tolist():
    if seqNum < 1654:
        continue
    row = df.loc[seqNum]
    alts.append(row['Alt'])
    azs.append(row['Az'])
    deltaAlt = row['Delta Alt Arcsec']
    deltaAz = row['Delta Az Arcsec']
    deltaAlts.append(deltaAlt)
    deltaAzs.append(deltaAz)
    print(f"{seqNum}: DeltaAlt = {deltaAlt:.1f} arcseconds, DeltaAz = {deltaAz:.1f} arcseconds")


In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle("StarTracker 20230309 Delta AltAz after header fix")
plt.subplots_adjust(hspace=0.3, wspace=0.3)
plt.subplot(2,2,1)
plt.scatter(alts, deltaAlts)
plt.xlabel('Alt')
plt.xlim(0,90)
plt.ylabel('Delta Alt arcsec')
plt.subplot(2,2,2)
plt.scatter(azs, deltaAlts)
plt.xlabel('Az')
plt.xlim(0,360)
plt.ylabel('Delta Alt arcsec')
plt.subplot(2,2,3)
plt.scatter(alts, deltaAzs)
plt.xlabel('Alt')
plt.xlim(0,90)
plt.ylabel('Delta Az arcsec')
plt.subplot(2,2,4)
plt.scatter(azs, deltaAzs)
plt.xlabel('Az')
plt.xlim(0,360)
plt.ylabel('Delta Az arcsec')
plt.savefig("/scratch/cslage/starTracker/Delta_AltAz_20230309.png")


In [ ]:
filename = '/scratch/cslage/starTracker/startracker_2023-03-07.json'
icrs = ICRS; fk5 = FK5(equinox='J2023')
df = pd.read_json(filename)
df = df.transpose()
df = df[~df.isnull().any(axis=1)]
alts = []
azs = []
deltaAlts = []
deltaAzs = []

for seqNum in df.index.values.tolist():
    row = df.loc[seqNum]
    time = Time(row['MJD'], format='mjd', scale='tai')
    skyLocation = SkyCoord(row['Ra']*u.deg, row['Dec']*u.deg, frame=icrs)
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
    obsAltAz1 = skyLocation.transform_to(altAz)
    deltaAlt = (row['Calculated Alt'] - obsAltAz1.alt.deg) * 3600.0
    deltaAz = (row['Calculated Az'] - obsAltAz1.az.deg) * 3600.0
    alts.append(row['Alt'])
    azs.append(row['Az'])
    deltaAlts.append(deltaAlt)
    deltaAzs.append(deltaAz)
    print(f"{seqNum}  DeltaAlt = {deltaAlt:.1f} arcseconds, DeltaAz = {deltaAz:.1f} arcseconds")

In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle("StarTracker 20230307 Delta AltAz Assuming:\nHeader RADec are in ICRS and translate those into AltAz to compare with anet solution")
plt.subplots_adjust(hspace=0.3, wspace=0.3)
plt.subplot(2,2,1)
plt.scatter(alts, deltaAlts)
plt.xlabel('Alt')
plt.xlim(0,90)
plt.ylabel('Delta Alt arcsec')
plt.subplot(2,2,2)
plt.scatter(azs, deltaAlts)
plt.xlabel('Az')
plt.xlim(0,360)
plt.ylabel('Delta Alt arcsec')
plt.subplot(2,2,3)
plt.scatter(alts, deltaAzs)
plt.xlabel('Alt')
plt.xlim(0,90)
plt.ylabel('Delta Az arcsec')
plt.subplot(2,2,4)
plt.scatter(azs, deltaAzs)
plt.xlabel('Az')
plt.xlim(0,360)
plt.ylabel('Delta Az arcsec')
plt.savefig("/scratch/cslage/starTracker/Delta_AltAz_20230307.png")


In [ ]:
df.columns